In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd

In [6]:
source_data = "data/train.json"
df = pd.read_json(source_data)

In [8]:
df.head(2)

,description,title,url,comments,sql,sql_plain,id
0,,Repent 151-160,http://data.stackexchange.com/stackoverflow/re...,[],"SELECT p.Title,p.score,p.ViewCount,p.AnswerCou...","SELECT \np.Title,p.score,p.ViewCount,p.AnswerC...",586912
1,,This is test data for my question to make thin...,http://data.stackexchange.com/stackoverflow/re...,[This is test data for my question to make thi...,"CREATE TABLE #CustomerDB ( ID int NOT NULL, NA...",-- This is test data for my question to make t...,54791


We only care about the *SQL* query for now.

In [17]:
sqls = df['sql']
sample = sqls.head()

The SQL dialect is very rich, to simplify the problem, we just care about the table relation for now.

i.e. `SELECT x.col1, x.col2 from table1 left join table2 on table1.key = table2.key`

I want to extract `table1` as the source table and it is related to `table2`

# sqlparse

Luckily, we can leverage existing library `sqlparse`, which helps parsing the dialect so that we don't have to write fancy regex

In [15]:
#hide
!pip install sqlparse

  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)


In [16]:
import sqlparse

In [46]:
sql = sample[0]

In [49]:
print(sqlparse.format(sql, reindent=True, keyword_case='upper'))

SELECT p.Title,
       p.score,
       p.ViewCount,
       p.AnswerCount,
       p.CommentCount,
       LEN(p.body) AS ques_size,
       p.favoritecount,
       p.id,
       u.reputation AS user_repo,
       u.creationdate AS user_join_date,
       datediff(MINUTE, p.CreationDate, a.CreationDate) AS QATimeGap
FROM posts AS p,
     posts AS a,
     users AS u
WHERE p.id=a.parentId
  AND p.owneruserid=u.id
  AND a.creationdate =
    (SELECT min(tau.creationdate)
     FROM posts tau
     WHERE tau.parentID=p.id
     GROUP BY tau.parentID)
  AND (p.title like '%Recommended method for handling UnsupportedEncodingException from String.getBytes("UTF-8")%'
       OR p.title like '%Way to format strings with "?" parameters to full string in java?%'
       OR p.title like '%How to parse a string without regular expressions%'
       OR p.title like '%How to replace string only once without regex in Java?%'
       OR p.title like '%Efficiently removing specific characters (some punctuation) from S

We can get the entity with `sqlparse`.

In [83]:
parsed = sqlparse.parse(sql)

In [85]:
parsed = parsed[0]

statement.tokens

[<DML 'SELECT' at 0x24B4F4453A8>,
 <Whitespace ' ' at 0x24B4F445E88>,
 <IdentifierList 'p.Titl...' at 0x24B50D3CAC8>,
 <Whitespace ' ' at 0x24B4EC78348>,
 <Keyword 'from' at 0x24B4EC783A8>,
 <Whitespace ' ' at 0x24B4EC78408>,
 <IdentifierList 'posts ...' at 0x24B50D3CCC8>,
 <Whitespace ' ' at 0x24B4EC78B88>,
 <Where 'where ...' at 0x24B4EC639C8>]

In [53]:
a=statement[0]

In [82]:
parsed

<Statement 'SELECT...' at 0x24B512FE8C8>

In [86]:
parsed.get_identifiers()

AttributeError: 'Statement' object has no attribute 'get_identifiers'

In [72]:
statement

<Statement 'SELECT...' at 0x24B4EC63448>

In [48]:
sqlparse.format(sql, reindent=True, keyword_case='upper')

'SELECT p.Title,\n       p.score,\n       p.ViewCount,\n       p.AnswerCount,\n       p.CommentCount,\n       LEN(p.body) AS ques_size,\n       p.favoritecount,\n       p.id,\n       u.reputation AS user_repo,\n       u.creationdate AS user_join_date,\n       datediff(MINUTE, p.CreationDate, a.CreationDate) AS QATimeGap\nFROM posts AS p,\n     posts AS a,\n     users AS u\nWHERE p.id=a.parentId\n  AND p.owneruserid=u.id\n  AND a.creationdate =\n    (SELECT min(tau.creationdate)\n     FROM posts tau\n     WHERE tau.parentID=p.id\n     GROUP BY tau.parentID)\n  AND (p.title like \'%Recommended method for handling UnsupportedEncodingException from String.getBytes("UTF-8")%\'\n       OR p.title like \'%Way to format strings with "?" parameters to full string in java?%\'\n       OR p.title like \'%How to parse a string without regular expressions%\'\n       OR p.title like \'%How to replace string only once without regex in Java?%\'\n       OR p.title like \'%Efficiently removing specific c